In [1]:
DURATION_YEAR_MAP = {
    "short": (1, 3),
    "medium": (4, 8),
    "long": (9, 20)
}

INVESTMENT_OPTIONS = [
    {
        "name": "Fixed Deposit",
        "risk": "Low",
        "duration": ["long", "medium"],
        "liquidity": "Medium",
        "tax_benefit": False,
        "goal_fit": ["capital_preservation", "retirement"],
        "user_type": ["conservative", "retiree"],
        "average_return_range": (0.06, 0.08),
        "allows_sip": False
    },
    {
        "name": "Public Provident Fund (PPF)",
        "risk": "Low",
        "duration": ["long"],
        "liquidity": "Low",
        "tax_benefit": True,
        "goal_fit": ["retirement", "tax_saving"],
        "user_type": ["conservative"],
        "average_return_range": (0.07, 0.075),
        "allows_sip": True
    },
    {
        "name": "Systematic Investment Plan (SIP - Equity MFs)",
        "risk": "Medium",
        "duration": ["medium", "long"],
        "liquidity": "High",
        "tax_benefit": False,
        "goal_fit": ["wealth_creation"],
        "user_type": ["moderate", "aggressive"],
        "average_return_range": (0.12, 0.16),
        "allows_sip": True
    },
    {
        "name": "Direct Equity (Stocks)",
        "risk": "High",
        "duration": ["medium", "long"],
        "liquidity": "High",
        "tax_benefit": False,
        "goal_fit": ["wealth_creation"],
        "user_type": ["aggressive", "moderate"],
        "average_return_range": (0.12, 0.18),
        "allows_sip": False
    },
    {
        "name": "NPS",
        "risk": "Medium",
        "duration": ["long"],
        "liquidity": "Low",
        "tax_benefit": True,
        "goal_fit": ["retirement"],
        "user_type": ["moderate", "conservative"],
        "average_return_range": (0.07, 0.10),
        "allows_sip": True
    },
    {
        "name": "Gold ETFs",
        "risk": "Medium",
        "duration": ["medium", "long"],
        "liquidity": "High",
        "tax_benefit": False,
        "goal_fit": ["hedge"],
        "user_type": ["moderate", "aggressive"],
        "average_return_range": (0.08, 0.12),
        "allows_sip": False
    },
    {
        "name": "Crypto Assets",
        "risk": "High",
        "duration": ["short", "medium"],
        "liquidity": "High",
        "tax_benefit": False,
        "goal_fit": ["wealth_creation"],
        "user_type": ["aggressive"],
        "average_return_range": (0.15, 0.30),
        "allows_sip": False
    },
    {
        "name": "Real Estate",
        "risk": "Medium",
        "duration": ["long"],
        "liquidity": "Low",
        "tax_benefit": False,
        "goal_fit": ["wealth_creation", "capital_preservation"],
        "user_type": ["moderate", "aggressive"],
        "average_return_range": (0.07, 0.12),
        "allows_sip": False
    },
    {
        "name": "Unit Linked Insurance Plan (ULIP)",
        "risk": "Medium",
        "duration": ["long"],
        "liquidity": "Low",
        "tax_benefit": True,
        "goal_fit": ["retirement", "tax_saving"],
        "user_type": ["moderate", "aggressive"],
        "average_return_range": (0.07, 0.12),
        "allows_sip": True
    }
]

PLAN_EXPLANATIONS = {
    "Fixed Deposit": "Safe, low-risk fixed returns with moderate liquidity. Good for conservative, income-focused investors.",
    "Public Provident Fund (PPF)": "Government-backed, tax-beneficial long-term saving, suited for retirement and cautious investors.",
    "Systematic Investment Plan (SIP - Equity MFs)": "Disciplined monthly equity investments for growth, suited to moderate-aggressive investors.",
    "Direct Equity (Stocks)": "High risk and return potential; suitable for aggressive/informed investors.",
    "NPS": "Balanced equity/debt retirement plan with tax benefits, fits moderate-conservative investors.",
    "Gold ETFs": "Invest in gold with good liquidity and moderate risk; hedge against inflation.",
    "Crypto Assets": "Very volatile but high-return potential; for aggressive investors only.",
    "Real Estate": "Capital appreciation and rental income over long term; low liquidity, higher investment size.",
    "Unit Linked Insurance Plan (ULIP)": "Insurance + investment combo offering moderate risk and tax benefits."
}

VALID_RISKS = ["conservative", "moderate", "aggressive"]
VALID_DURATIONS = ["short", "medium", "long"]
VALID_GOALS = ["retirement", "wealth_creation", "capital_preservation", "tax_saving", "hedge", "protection"]


def get_float_input(prompt):
    while True:
        try:
            val = input(prompt)
            val_float = float(val)
            if val_float < 0:
                print("Value cannot be negative. Please enter a valid amount.")
                continue
            return val_float
        except ValueError:
            print("Invalid number. Please enter a valid numeric value.")


def get_validated_input(prompt, valid_choices):
    while True:
        val = input(prompt).strip().lower()
        if val in valid_choices:
            return val
        print(f"Invalid choice! Please choose from {valid_choices}.")


def get_multiple_valid_inputs(prompt, valid_choices):
    while True:
        user_input = input(prompt).strip().lower()
        inputs = [x.strip() for x in user_input.split(",") if x.strip()]
        invalid = [x for x in inputs if x not in valid_choices]
        if invalid:
            print(f"Invalid choice(s): {invalid}. Please choose from {valid_choices}.")
        elif not inputs:
            print("You must provide at least one valid choice.")
        else:
            return inputs


def map_existing_investments(user_input_list, investment_options):
    mapped_investments = []
    option_names = [inv["name"].lower() for inv in investment_options]
    abbreviation_map = {
        "sip": "systematic investment plan (sip - equity mfs)",
        "ppf": "public provident fund (ppf)",
        "fd": "fixed deposit",
        "nps": "nps",
        "ulip": "unit linked insurance plan (ulip)",
        "re": "real estate",
        "crypto": "crypto assets",
        "direct equity": "direct equity (stocks)",
        "health insurance": "health insurance"
    }
    for user_inv in user_input_list:
        user_inv_lower = user_inv.lower()
        if user_inv_lower in abbreviation_map:
            mapped_investments.append(abbreviation_map[user_inv_lower])
            continue
        matches = difflib.get_close_matches(user_inv_lower, option_names, n=1, cutoff=0.5)
        mapped_investments.append(matches[0] if matches else user_inv_lower)
    return mapped_investments


def get_duration_years(label):
    start, end = DURATION_YEAR_MAP[label]
    avg = (start + end) // 2
    return avg, (start, end)


def compute_lump_sum_range(principal, rate_range, years):
    low = principal * (1 + rate_range[0]) ** years
    high = principal * (1 + rate_range[1]) ** years
    return round(low, 2), round(high, 2)


def compute_sip_range(monthly, rate_range, years):
    n = years * 12
    r_low = rate_range[0] / 12
    r_high = rate_range[1] / 12
    fv_low = monthly * (((1 + r_low) ** n - 1) / r_low) * (1 + r_low)
    fv_high = monthly * (((1 + r_high) ** n - 1) / r_high) * (1 + r_high)
    return round(fv_low, 2), round(fv_high, 2)


def suggest_investments(user, options):
    recommendations, restrictions, warnings = [], [], []
    age = user["age"]
    annual_surplus = (user["income"] - user["expenses"] - user["emi"]) * 12
    emergency_required = user["expenses"] * 6
    orig_risk = user["risk_appetite"]
    if age < 30:
        user["risk_appetite"] = "aggressive"
    elif 30 <= age <= 45 and orig_risk == "aggressive":
        user["risk_appetite"] = "moderate"
    elif age > 45:
        user["risk_appetite"] = "conservative"
    if orig_risk != user["risk_appetite"]:
        warnings.append(
            f"Your risk profile adjusted from {orig_risk} to {user['risk_appetite']} due to age {age}."
        )
    if annual_surplus < emergency_required:
        recommendations.append("Emergency Fund (6 months expenses)")
        warnings.append("Savings insufficient for emergencies.")
    if not user["insurance_coverage"]:
        recommendations.append("Health Insurance")
        warnings.append("You lack insurance; health insurance recommended.")

    for option in options:
        name_lower = option["name"].lower()
        if any(existing.lower() == name_lower for existing in user["existing_portfolio"]):
            restrictions.append(f"You already have {option['name']} investment.")
            continue
        if user["risk_appetite"] not in [p.lower() for p in option["user_type"]]:
            restrictions.append(f"{option['name']} suited for {', '.join(option['user_type'])} risk profiles.")
            continue
        if user["investment_duration"] not in option["duration"]:
            restrictions.append(f"{option['name']} best for {', '.join(option['duration'])} durations.")
            continue
        if not any(goal in option["goal_fit"] for goal in user["goals"]):
            restrictions.append(f"{option['name']} fits goals: {', '.join([g.replace('_',' ').title() for g in option['goal_fit']])}.")
            continue
        if user["needs_tax_benefit"] and not option["tax_benefit"]:
            restrictions.append(f"{option['name']} has no tax benefits.")
            continue
        if user["mode"] == "monthly" and not option["allows_sip"]:
            restrictions.append(f"{option['name']} does not support monthly investments.")
            continue
        recommendations.append(option["name"])
    if not recommendations:
        warnings.append("No suitable investment options found; consider revising your inputs.")
    return recommendations, restrictions, warnings


def format_money(val):
    return f"₹{val:,.2f}"


def get_plan_explanation(plan_name):
    return PLAN_EXPLANATIONS.get(plan_name, "No explanation available.")


def get_user_input():
    print("Welcome to the Investment Recommendation Tool!")
    mode = ""
    while mode not in ["lump", "monthly"]:
        mode = input("Do you want to invest as a lump sum or monthly? (Enter lump/monthly): ").strip().lower()
    amount_prompt = "Enter the lump sum amount (₹): " if mode == "lump" else "Enter the amount you want to invest monthly (₹): "
    amount = get_float_input(amount_prompt)
    age = int(get_float_input("Enter your age: "))
    income = get_float_input("Enter your monthly income (₹): ")
    expenses = get_float_input("Enter your monthly expenses (₹): ")
    emi = get_float_input("Enter your monthly EMI payments (₹): ")
    debts = get_float_input("Enter your total personal debts (₹): ")
    risk_appetite = get_validated_input(
        "Risk appetite (conservative, moderate, aggressive): ", VALID_RISKS)
    duration = get_validated_input(
        "Investment duration (short, medium, long): ", VALID_DURATIONS)
    goals = get_multiple_valid_inputs(
        "Enter your financial goals (comma separated): ", VALID_GOALS)
    tax_pref = get_validated_input("Do you prefer tax saving investments? (yes/no): ", ["yes", "no"])
    tax_benefit = tax_pref == "yes"
    insurance_cov = get_validated_input("Do you have sufficient health/life insurance? (yes/no): ", ["yes", "no"])
    insurance_coverage = insurance_cov == "yes"
    existing_input = input("Enter existing investments (comma separated), or leave blank: ").strip()
    existing_portfolio = []
    if existing_input:
        existing_list = [x.strip() for x in existing_input.split(",") if x.strip()]
        existing_portfolio = map_existing_investments(existing_list, INVESTMENT_OPTIONS)
    return {
        "mode": mode,
        "invest_amt": amount if mode == "lump" else None,
        "sip_amt": amount if mode == "monthly" else None,
        "age": age,
        "income": income,
        "expenses": expenses,
        "emi": emi,
        "personal_debt": debts,
        "risk_appetite": risk_appetite,
        "investment_duration": duration,
        "goals": goals,
        "needs_tax_benefit": tax_benefit,
        "insurance_coverage": insurance_coverage,
        "existing_portfolio": existing_portfolio
    }


def main():
    user_profile = get_user_input()
    recommendations, restrictions, warnings = suggest_investments(user_profile, INVESTMENT_OPTIONS)
    avg_years, (min_year, max_year) = get_duration_years(user_profile["investment_duration"])

    print("\n-- Your Investment Recommendations --\n")
    print(
        f"Investment amount ({user_profile['mode']}): ₹{user_profile['invest_amt'] or user_profile['sip_amt']}"
    )
    print(
        f"Investment Duration: {user_profile['investment_duration'].title()} ({min_year}-{max_year} years)\n"
    )

    if recommendations:
        for idx, name in enumerate(recommendations, 1):
            opt = next((o for o in INVESTMENT_OPTIONS if o["name"] == name), None)
            if not opt:
                continue
            low_rate, high_rate = opt["average_return_range"]
            if user_profile["mode"] == "monthly":
                low_val, high_val = compute_sip_range(
                    user_profile["sip_amt"], opt["average_return_range"], avg_years
                )
                formula = "Future Value of Monthly Investments"
            else:
                low_val, high_val = compute_lump_sum_range(
                    user_profile["invest_amt"], opt["average_return_range"], avg_years
                )
                formula = "Compounded Future Value of Lump Sum Investment"
            print(f"{idx}. {opt['name']}")
            print(f"   Risk: {opt['risk']}")
            print(f"   Annual Return Range: {low_rate*100:.1f}% - {high_rate*100:.1f}%")
            print(f"   Projected Value in {avg_years} years: {format_money(low_val)} - {format_money(high_val)} [{formula}]")
            print(f"   Suitable for: {', '.join([g.replace('_',' ').title() for g in opt['goal_fit']])}")
            print(f"   Explanation: {get_plan_explanation(opt['name'])}\n")
    else:
        print("No suitable investment options found based on your inputs.")

    if restrictions:
        print("\nFollowing options were skipped due to your selections:")
        for r in restrictions:
            print(f"- {r}")

    if warnings:
        print("\nWarnings/Suggestions:")
        for w in warnings:
            print(f"- {w}")


if __name__ == "__main__":
    main()


Welcome to the Investment Recommendation Tool!


Do you want to invest as a lump sum or monthly? (Enter lump/monthly):  monthly
Enter the amount you want to invest monthly (₹):  40000
Enter your age:  25
Enter your monthly income (₹):  120000
Enter your monthly expenses (₹):  25000
Enter your monthly EMI payments (₹):  5000
Enter your total personal debts (₹):  40000
Risk appetite (conservative, moderate, aggressive):  aggressive
Investment duration (short, medium, long):  medium
Enter your financial goals (comma separated):  wealth_creation
Do you prefer tax saving investments? (yes/no):  no
Do you have sufficient health/life insurance? (yes/no):  yes
Enter existing investments (comma separated), or leave blank:  



-- Your Investment Recommendations --

Investment amount (monthly): ₹40000.0
Investment Duration: Medium (4-8 years)

1. Systematic Investment Plan (SIP - Equity MFs)
   Risk: Medium
   Annual Return Range: 12.0% - 16.0%
   Projected Value in 6 years: ₹4,230,281.22 - ₹4,849,350.73 [Future Value of Monthly Investments]
   Suitable for: Wealth Creation
   Explanation: Disciplined monthly equity investments for growth, suited to moderate-aggressive investors.


Following options were skipped due to your selections:
- Fixed Deposit suited for conservative, retiree risk profiles.
- Public Provident Fund (PPF) suited for conservative risk profiles.
- Direct Equity (Stocks) does not support monthly investments.
- NPS suited for moderate, conservative risk profiles.
- Gold ETFs fits goals: Hedge.
- Crypto Assets does not support monthly investments.
- Real Estate best for long durations.
- Unit Linked Insurance Plan (ULIP) best for long durations.


In [3]:
import difflib
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Investment Recommendation Dashboard"

# Prepare options for dropdowns similar as in CLI version
DEFAULT_RISK = "moderate"
DEFAULT_DURATION = "medium"
DEFAULT_GOALS = []

risk_options = [{"label": r.title(), "value": r} for r in ["conservative", "moderate", "aggressive"]]
duration_options = [{"label": f"{d.title()} ({d_range[0]}-{d_range[1]} yrs)", "value": d} for d, d_range in {
    "short": (1, 3),
    "medium": (4, 8),
    "long": (9, 20)
}.items()]
goal_options = [{"label": g.replace("_", " ").title(), "value": g} for g in ["retirement", "wealth_creation", "capital_preservation", "tax_saving", "hedge", "protection"]]

app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H2("Investment Recommendation Tool"), className="text-center my-4")),
    dbc.Row([
        dbc.Col([
            dbc.Label("Investment Mode"),
            dcc.RadioItems(
                id="mode",
                options=[
                    {"label": "Lump Sum", "value": "lump"},
                    {"label": "Monthly (SIP)", "value": "monthly"}
                ],
                value="lump",
                inline=True,
            ),
            dbc.Label("Investment Amount (₹)"),
            dcc.Input(id="amount", type="number", placeholder="Enter amount", min=0, required=True, style={"width": "100%"}),
            dbc.Label("Age"),
            dcc.Input(id="age", type="number", min=0, required=True, style={"width": "100%"}),
            dbc.Label("Monthly Income (₹)"),
            dcc.Input(id="income", type="number", min=0, required=True, style={"width": "100%"}),
            dbc.Label("Monthly Expenses (₹)"),
            dcc.Input(id="expenses", type="number", min=0, required=True, style={"width": "100%"}),
            dbc.Label("Monthly EMI Payments (₹)"),
            dcc.Input(id="emi", type="number", min=0, required=True, style={"width": "100%"}),
            dbc.Label("Total Personal Debts (₹)"),
            dcc.Input(id="debts", type="number", min=0, required=True, style={"width": "100%"}),
            dbc.Label("Risk Appetite"),
            dcc.Dropdown(id="risk_appetite", options=risk_options, value=DEFAULT_RISK, clearable=False),
            dbc.Label("Investment Duration"),
            dcc.Dropdown(id="duration", options=duration_options, value=DEFAULT_DURATION, clearable=False),
            dbc.Label("Financial Goals"),
            dcc.Dropdown(id="goals", options=goal_options, value=DEFAULT_GOALS, multi=True),
            dbc.Label("Prefer Tax Saving Investments"),
            dcc.RadioItems(
                id="tax_saving",
                options=[{"label": "Yes", "value": True}, {"label": "No", "value": False}],
                value=False,
                inline=True
            ),
            dbc.Label("Sufficient Health/Life Insurance"),
            dcc.RadioItems(
                id="insurance",
                options=[{"label": "Yes", "value": True}, {"label": "No", "value": False}],
                value=True,
                inline=True,
            ),
            dbc.Label("Existing Investments (comma-separated)"),
            dcc.Input(id="existing", type="text", placeholder="e.g. PPF, FD, SIP", style={"width": "100%"}),
            dbc.Button("Get Recommendations", id="submit-btn", color="primary", className="mt-3", n_clicks=0),
            html.Div(id="warnings", style={"color": "orange", "marginTop": "10px"}),
            html.Div(id="restrictions", style={"color": "red", "marginTop": "10px"}),
        ], md=5),
        dbc.Col([
            html.H4("Investment Recommendations"),
            html.Div(id="recommendations"),
        ], md=7),
    ]),
], fluid=True)

@app.callback(
    [
        Output("recommendations", "children"),
        Output("warnings", "children"),
        Output("restrictions", "children"),
    ],
    [
        Input("submit-btn", "n_clicks"),
    ],
    [
        State("mode", "value"),
        State("amount", "value"),
        State("age", "value"),
        State("income", "value"),
        State("expenses", "value"),
        State("emi", "value"),
        State("debts", "value"),
        State("risk_appetite", "value"),
        State("duration", "value"),
        State("goals", "value"),
        State("tax_saving", "value"),
        State("insurance", "value"),
        State("existing", "value"),
    ],
)
def update_recommendations(n_clicks, mode, amount, age, income, expenses, emi, debts, risk_appetite, duration, goals, tax_saving, insurance, existing):
    if n_clicks == 0:
        return "", "", ""

    # Basic validation
    missing = []
    for v, name in zip(
        [amount, age, income, expenses, emi, debts, risk_appetite, duration, goals],
        ["Amount", "Age", "Income", "Expenses", "EMI", "Debts", "Risk Appetite", "Duration", "Goals"],
    ):
        if v is None or (isinstance(v, list) and len(v) == 0):
            missing.append(name)

    if missing:
        return [html.Div(f"Please provide values for: {', '.join(missing)}", style={"color": "red"})], "", ""

    user = {
        "mode": mode,
        "invest_amt": amount if mode == "lump" else None,
        "sip_amt": amount if mode == "monthly" else None,
        "age": age,
        "income": income,
        "expenses": expenses,
        "emi": emi,
        "personal_debt": debts,
        "risk_appetite": risk_appetite,
        "investment_duration": duration,
        "goals": goals if isinstance(goals, list) else [goals],
        "needs_tax_benefit": tax_saving,
        "insurance_coverage": insurance,
        "existing_portfolio": [],
    }

    if existing and existing.strip():
        existing_list = [e.strip() for e in existing.split(",") if e.strip()]
        user["existing_portfolio"] = map_existing_investments(existing_list, INVESTMENT_OPTIONS)

    recommendations, restrictions, warnings = suggest_investments(user, INVESTMENT_OPTIONS)
    avg_years, (min_y, max_y) = get_duration_years(duration)

    if recommendations:
        rec_display = []
        for idx, name in enumerate(recommendations, 1):
            opt = next((it for it in INVESTMENT_OPTIONS if it["name"] == name), None)
            if not opt:
                continue
            low, high = opt["average_return_range"]
            if user["mode"] == "monthly":
                val_low, val_high = compute_sip_range(user["sip_amt"], opt["average_return_range"], avg_years)
                formula = "Future Value of Monthly Investments"
            else:
                val_low, val_high = compute_lump_sum_range(user["invest_amt"], opt["average_return_range"], avg_years)
                formula = "Compounded Future Value of Lump Sum Investment"

            rec_display.append(
                html.Div(
                    dbc.Card(
                        dbc.CardBody(
                            [
                                html.H5(f"{idx}. {opt['name']}"),
                                html.P(
                                    [
                                        f"Risk: {opt['risk']}",
                                        html.Br(),
                                        f"Annual Return Range: {low*100:.1f}% - {high*100:.1f}%",
                                        html.Br(),
                                        f"Projected value in {avg_years} years: {val_low:,.2f} - {val_high:,.2f} ₹",
                                        html.Br(),
                                        f"{formula}",
                                        html.Br(),
                                        f"Suitable for: {', '.join([g.replace('_',' ').title() for g in opt['goal_fit']])}",
                                        html.Br(),
                                        f"Explanation: {get_plan_explanation(opt['name'])}",
                                    ]
                                ),
                            ]
                        ),
                        className="mb-3 shadow",
                    )
                )
            )
    else:
        rec_display = html.Div("No suitable investment options found based on your inputs.")

    return rec_display, "\n".join(warnings), "\n".join(restrictions)


if __name__ == "__main__":
    app.run(debug=True)
